## Deliverable 2: DISTRIBUTED - Distributed implementation (Due 14th october 2024)

After that, you need to parallelize that code using the native support for Distributed Training in Pytorch or Lightning. The distributed training tool (native or Lightning) and the strategy (DP, DDP, Zero, FSDP) for work distribution depend on you. You can actually experience several if you feel like it.

Once you have the implementation, you have to test it and measure execution times as you did with the BASELINE. Report just times or the output of a profiling time as you did before.

In [1]:
!nvidia-smi

Mon Oct 14 23:17:53 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.54.03              Driver Version: 535.54.03    CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-PCIE-40GB          On  | 00000000:17:00.0 Off |                    0 |
| N/A   33C    P0              34W / 250W |      4MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [2]:
# Install required libraries
!pip install transformers datasets torch

You should consider upgrading via the '/mnt/netapp2/Store_uni/home/ulc/cursos/curso355/mypython/bin/python3 -m pip install --upgrade pip' command.


In [3]:
# Import necessary libraries
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, Trainer, TrainingArguments
from datasets import load_dataset
from transformers import default_data_collator

2024-10-14 23:17:58.461034: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-14 23:17:58.600304: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-14 23:17:59.583721: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /mnt/netapp1/Optcesga_FT2_RHEL7/2020/gentoo/22072020/usr/gr/lib64
2024-10-14 23:17:59.5

In [4]:
# Load SQuAD dataset
squad_dataset = load_dataset('squad')

# Load pre-trained BERT tokenizer and model
# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
model = AutoModelForQuestionAnswering.from_pretrained('bert-base-uncased')

Found cached dataset parquet (/home/ulc/cursos/curso355/.cache/huggingface/datasets/parquet/plain_text-7d14fdbf55ecc00c/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)


  0%|          | 0/2 [00:00<?, ?it/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForQuestionAnswering: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_out

In [5]:
def prepare_train_features(examples):
    # Tokenize our examples with truncation and padding, but keep the overflows using a stride.
    # This results in one example possible giving several features when a context is long,
    # each of those features having a context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question"],
        examples["context"],
        truncation="only_second",  # truncate context, not the question
        max_length=384,
        stride=128,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context.
    # This will help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != 1:
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != 1:
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [8]:

# Apply the function to our data
tokenized_datasets = squad_dataset.map(prepare_train_features, batched=True, remove_columns=squad_dataset["train"].column_names)

Loading cached processed dataset at /home/ulc/cursos/curso355/.cache/huggingface/datasets/parquet/plain_text-7d14fdbf55ecc00c/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7/cache-aff7705da5561f56.arrow
Loading cached processed dataset at /home/ulc/cursos/curso355/.cache/huggingface/datasets/parquet/plain_text-7d14fdbf55ecc00c/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7/cache-1296e189dc3ae491.arrow


In [11]:
# Define training arguments for distributed training
training_args = TrainingArguments(
    output_dir="./results",               # Output directory for model checkpoints
    evaluation_strategy="epoch",          # Evaluate after each epoch
    learning_rate=3e-5,                   # Learning rate
    per_device_train_batch_size=16,       # Batch size per device (per GPU)
    per_device_eval_batch_size=16,        # Batch size for evaluation per device
    num_train_epochs=3,                   # Number of training epochs
    weight_decay=0.01,                    # Weight decay to prevent overfitting
    logging_dir="./logs",                 # Directory for storing logs
    logging_steps=10,                     # Log every 10 steps
    fp16=True,                            # Enable mixed precision training
    report_to="none",                     # Disable reporting to any external platforms (like WandB)
    dataloader_num_workers=4,             # Number of workers for data loading
    gradient_accumulation_steps=1,        # Accumulate gradients before optimizing
    sharded_ddp="simple",                 # Enable simple distributed data parallelism (to shard the optimizer states)
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=default_data_collator
)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:25                                                                                   │
│                                                                                                  │
│   22 │   args=training_args,                                                                     │
│   23 │   train_dataset=tokenized_datasets["train"],                                              │
│   24 │   eval_dataset=tokenized_datasets["validation"],                                          │
│ ❱ 25 │   tokenizer=tokenizer,                                                                    │
│   26 )                                                                                           │
│   27                                                                                             │
│                                                                                                  │
│ /mnt/netapp2/Store_uni/home/ulc/cursos/curso355/mypython/lib/python3.7/site-packages/transformer │
│ s/trainer.py:431 in __init__                                                                     │
│                                                                                                  │
│    428 │   │   │   │   │   "Using --sharded_ddp xxx together with --fsdp is not possible, deact  │
│    429 │   │   │   │   )                                                                         │
│    430 │   │   │   if args.parallel_mode != ParallelMode.DISTRIBUTED:                            │
│ ❱  431 │   │   │   │   raise ValueError("Using sharded DDP only works in distributed training."  │
│    432 │   │   │   elif not is_fairscale_available():                                            │
│    433 │   │   │   │   raise ImportError("Sharded DDP training requires fairscale: `pip install  │
│    434 │   │   │   elif ShardedDDPOption.SIMPLE not in args.sharded_ddp and FullyShardedDDP is   │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ValueError: Using sharded DDP only works in distributed training.

In [ ]:
# Train the model
trainer.train()

In [ ]:
# Evaluate the model
trainer.evaluate()